<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section5/section5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# API KEYの設定

In [1]:
%%capture
!pip install openai==1.3.5 httpx==0.27.2 --force-reinstall

In [2]:
from typing import List, Dict, Tuple

import getpass
apikey = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください')

from openai import OpenAI
client = OpenAI(api_key=apikey)

OpenAIのAPIキーを入力してください··········


# プロンプトを入力して返答を返す関数を定義

In [3]:
def get_chatgpt_response(
    user_input: str,
    template: str,
    model: str = "gpt-3.5-turbo-1106",
    temperature: float = 0,
    max_tokens: int = 500,
) -> str:
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(user_input=user_input)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        seed=0
    )
    return response.choices[0].message.content

In [4]:
PROMPT_TEMPLATE = """
    下記の文章を関西弁にしてください。
    {user_input}
    """

In [5]:
print(get_chatgpt_response("がむしゃらにやれ！", PROMPT_TEMPLATE))

がんばってやりなさいやで！


In [6]:
print(get_chatgpt_response("がむしゃらにやるな！", PROMPT_TEMPLATE))

がんばってやるな！


# 問い合わせ分類

**問い合わせが以下のどのカテゴリーか分類させよう**
  - 製品に対する問い合わせ
  - 注文に対する問い合わせ
  - 配送に対する問い合わせ

まずは、プロンプトのテンプレートを定義

In [7]:
PROMPT_TEMPLATE = """
    あなたは優秀なカスタマーサポートセンターの担当者です。
    下記顧客からの問い合わせと分類カテゴリーがあります。
    顧客からの問い合わせ内容がどのカテゴリーに当てはまるか考えてください。
    可能な限りカテゴリー分けしてください。それでも無い場合「該当なし」としてください。

    # 分類カテゴリー
     製品に対する問い合わせ
     注文に対する問い合わせ
     配送に対する問い合わせ

    # 顧客からの問い合わせ
    {user_input}

    # 出力形式
     製品に対する問い合わせ, 注文に対する問い合わせ, 配送に対する問い合わせ のどれか
    """

In [8]:
get_chatgpt_response("本日購入した場合、最短で納期はいつですか？", PROMPT_TEMPLATE)

'配送に対する問い合わせ'

In [9]:
get_chatgpt_response("購入をキャンセルしたいです", PROMPT_TEMPLATE)

'注文に対する問い合わせ'

In [10]:
get_chatgpt_response("どのような商品がありますか？", PROMPT_TEMPLATE)

'製品に対する問い合わせ'

In [11]:
get_chatgpt_response("猫を飼いたいです", PROMPT_TEMPLATE)

'該当なし'

# 問い合わせの詳細カテゴリー

問い合わせの内容により、以下のカテゴリーに分類します。

## 製品に対する問い合わせ
  - **スペック**
  - **値段**
  - **購入方法**

## 注文に対する問い合わせ
  - **注文状況**
  - **注文変更**
  - **支払い方法**

## 配送に対する問い合わせ
  - **配送状況**
  - **配送オプション**
  - **配送問題**


In [ ]:
PROMPT_TEMPLATE = """
    あなたは優秀なカスタマーサポートセンターの担当者です。
    下記顧客からの問い合わせと分類カテゴリーがあります。
    次のステップで段階的に顧客の問い合わせを分類してください。
    1. 顧客からの問い合わせ内容を第一カテゴリーに分ける
    2. 第一カテゴリーに紐づいている最も近い第二カテゴリーがどれかを考える
    3. 出力形式に従って文章を生成する

    # 顧客からの問い合わせ
    {user_input}

    # 分類カテゴリー
      第一カテゴリー：
        製品, 注文, 配送
      第二カテゴリー：
        製品の場合は[スペック, 値段, 購入方法]のどれか
        注文の場合は[注文状況, 注文変更, 支払い方法]のどれか
        配送の場合は[配送状況, 配送オプション, 配送問題]のどれか

    # 出力形式
      第一カテゴリー:<第一カテゴリーのどれか>
      第二カテゴリー:<第二カテゴリーのどれか>
    """

In [ ]:
response = get_chatgpt_response("私のパッケージは出荷されましたか？", PROMPT_TEMPLATE)
print(response)

    第一カテゴリー: 配送
    第二カテゴリー: 配送状況


In [ ]:
response = get_chatgpt_response("到着日時を変更して欲しいです", PROMPT_TEMPLATE)
print(response)

第一カテゴリー: 配送
      第二カテゴリー: 配送オプション


In [ ]:
response = get_chatgpt_response("商品が微妙です", PROMPT_TEMPLATE)
print(response)

第一カテゴリー: 製品
      第二カテゴリー: スペック


In [ ]:
print(response.replace(" ", ""))

第一カテゴリー:製品
第二カテゴリー:スペック


# JSONモード

In [12]:
def get_chatgpt_response(
    user_input: str,
    template: str,
    model: str = "gpt-3.5-turbo-1106",
    temperature: float = 0,
    max_tokens: int = 500,
) -> str:
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(user_input=user_input)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        response_format={ "type": "json_object" }, #この設定により、出力は必ずJSON形式となる
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        seed=0
    )
    return response.choices[0].message.content

In [13]:
PROMPT_TEMPLATE = """
    あなたは優秀なカスタマーサポートセンターの担当者です。
    下記顧客からの問い合わせと分類カテゴリーがあります。
    次のステップで段階的に顧客の問い合わせを分類してください。
    1. 顧客からの問い合わせ内容を第一カテゴリーに分ける
    2. 第一カテゴリーに紐づいている最も近い第二カテゴリーがどれかを考える
    3. JSON形式(key_1:<第一カテゴリーのどれか>, key_2:<第二カテゴリーのどれか>)で出力する
    可能な限りカテゴリー分けしてください。それでも無い場合カテゴリーのところを「該当なし」としてください。
    例：key_1:該当なし, key_2:該当なし

    # 顧客からの問い合わせ
    {user_input}

    # 分類カテゴリー
      第一カテゴリー：
        製品, 注文, 配送
      第二カテゴリー：
        製品の場合は[スペック, 値段, 購入方法]のどれか
        注文の場合は[注文状況, 注文変更, 支払い方法]のどれか
        配送の場合は[配送状況, 配送オプション, 配送問題]のどれか

    # 出力形式
     {{"key_1": "製品", "key_2": "値段"}}
    """

In [14]:
print(get_chatgpt_response("このスマートフォンは重たいですか？", PROMPT_TEMPLATE))

{"key_1": "製品", "key_2": "スペック"}


# 問い合わせ内容が膨大にある
  - "このスマートフォンのプロセッサーは何ですか？"
  - "このパソコンのRAMの容量はどれくらいですか？"
  - "このカメラの最大シャッタースピードは何ですか？"
  - "このヘッドフォンの周波数レンジは何ですか？"
  - "この自転車のフレームは何で作られていますか？"
  - "このスマートフォンの最新の価格は何ですか？"
  - "このゲーム機の割引価格はありますか？"
  - "この製品の月々の費用はどれくらいですか？"
  - "このサブスクリプションサービスの年間費用は何ですか？"
  - "この洗濯機には何年間の保証が含まれていますか？そのコストはどれくらいですか？"
  - "この製品はオンラインで注文することができますか？"
  - "この製品を店舗で購入することは可能ですか？"
  - "この製品は分割払いで購入できますか？"
  - "この製品は海外から注文できますか？"
  - "この製品を予約注文することは可能ですか？"
  - "私の注文はすでに確認されましたか？"
  - "私の注文はまだ出荷されていませんか？"
  - "注文した商品の在庫状況を教えてください。"
  - "注文の進行状況を確認するにはどうすればよいですか？"
  - "私の注文はキャンセルされましたか？"
  - "私の注文の配送先住所を変更することは可能ですか？"
  - "注文した商品の色やサイズを変更することは可能ですか？"
  - "注文した商品を別のものに変更することは可能ですか？"
  - "注文の数量を増やすことは可能ですか？"  
  - "注文した商品をキャンセルすることは可能ですか？"
  - "注文の支払いにクレジットカードは使用できますか？"
  - "注文の支払いにデビットカードは使用できますか？"
  - "注文の支払いにPayPalは使用できますか？"
  - "注文の支払いにApple PayやGoogle Payは使用できますか？"
  - "注文の支払いにビットコインは使用できますか？"
  - "私のパッケージは出荷されましたか？"
  - "私のパッケージの追跡番号を教えてください。"
  - "私のパッケージはいつ到着予定ですか？"
  - "私のパッケージが遅延しています。なぜですか？"
  - "私のパッケージは配送中に失われたようです。どうすればいいですか？"
  - "エクスプレス配送は可能ですか？"
  - "特定の日時に配送することはできますか？"
  - "配送先を注文後に変更することは可能ですか？"
  - "商品の配送はどの運送会社を利用していますか？"
  - "私の地域への配送は可能ですか？"
  - "配送された商品が損傷していました。どうすればいいですか？"
  - "配送された商品が間違っていました。どうすればいいですか？"
  - "パッケージが届かない場合、どうすればいいですか？"
  - "配達員が不在票を残していました。どうすればいいですか？"
  - "パッケージが開封されていました。どうすればいいですか？"

In [15]:
inquiries = [
    "このカメラの最大シャッタースピードは何ですか？",
    "私のパッケージの追跡番号を教えてください。",
    "エクスプレス配送は可能ですか？",
    "特定の日時に配送することはできますか？",
    "配送された商品が損傷していました。どうすればいいですか？",
    "注文の支払いにApple PayやGoogle Payは使用できますか？",
    "私のパッケージはいつ到着予定ですか？",
    "注文の支払いにクレジットカードは使用できますか？",
    "私の注文はまだ出荷されていませんか？",
    "この製品はオンラインで注文することができますか？",
    "私のパッケージが遅延しています。なぜですか？",
    "このスマートフォンの最新の価格は何ですか？",
    "注文した商品をキャンセルすることは可能ですか？",
    "商品の配送はどの運送会社を利用していますか？",
    "このサブスクリプションサービスの年間費用は何ですか？",
    "このヘッドフォンの周波数レンジは何ですか？",
    "私の地域への配送は可能ですか？",
    "注文の支払いにビットコインは使用できますか？",
    "配送された商品が間違っていました。どうすればいいですか？",
    "この製品を店舗で購入することは可能ですか？",
    "パッケージが届かない場合、どうすればいいですか？",
    "私の注文はキャンセルされましたか？",
    "このパソコンのRAMの容量はどれくらいですか？",
    "注文の支払いにデビットカードは使用できますか？",
    "私のパッケージは出荷されましたか？",
    "注文の数量を増やすことは可能ですか？",
    "私の注文はすでに確認されましたか？",
    "注文の進行状況を確認するにはどうすればよいですか？",
    "配送先を注文後に変更することは可能ですか？",
    "パッケージが開封されていました。どうすればいいですか？",
    "配達員が不在票を残していました。どうすればいいですか？",
    "私のパッケージは配送中に失われたようです。どうすればいいですか？",
    "この自転車のフレームは何で作られていますか？",
    "この製品を予約注文することは可能ですか？",
    "注文した商品の色やサイズを変更することは可能ですか？",
    "この製品の月々の費用はどれくらいですか？",
    "このスマートフォンのプロセッサーは何ですか？",
    "この製品は分割払いで購入できますか？",
    "この製品は海外から注文できますか？",
    "注文した商品の在庫状況を教えてください。",
    "このゲーム機の割引価格はありますか？",
    "私の注文の配送先住所を変更することは可能ですか？",
    "注文した商品を別のものに変更することは可能ですか？",
    "この洗濯機には何年間の保証が含まれていますか？そのコストはどれくらいですか？",
    "注文の支払いにPayPalは使用できますか？",
]

In [16]:
from tqdm import tqdm
import json


def get_responses_and_check_json(inquiries: List[str]) -> List[str]:
    """
    ChatGPTからの応答を取得し、それらがJSON形式であることを確認する
    """
    list_json = []

    def json_checker(response):
        try:
            return json.loads(response.strip())
        except json.JSONDecodeError:
            print(f"No json format... {response.strip()}") # response_formatを指定しているので、JSON以外が出力されることはない。その確認用のもの

    for inquery in tqdm(inquiries):
        response = get_chatgpt_response(inquery, PROMPT_TEMPLATE)
        list_json.append(json_checker(response))

    return list_json

In [17]:
list_json = get_responses_and_check_json(inquiries)

100%|██████████| 45/45 [00:25<00:00,  1.79it/s]


In [18]:
import pandas as pd

df = pd.DataFrame(list_json)
df["inquiry"] = inquiries
df.columns = ["category_1", "category_2", "inquiry"]

In [19]:
df["category_1"].value_counts(normalize=True) * 100

,proportion
category_1,
製品,33.333333
配送,33.333333
注文,33.333333


In [20]:
df["category_2"].value_counts(normalize=True) * 100

,proportion
category_2,
配送問題,17.777778
スペック,11.111111
配送状況,11.111111
該当なし,11.111111
支払い方法,11.111111
値段,11.111111
注文変更,8.888889
注文状況,6.666667
購入方法,6.666667


# 各カテゴリーに対応した返答文の生成

問い合わせカテゴリーごとに、適切な問い合わせ先を示す返答文を作成しましょう。

## シチュエーション

具体的には、ユーザーからの問い合わせを適切に分類し、その問い合わせの内容に応じた連絡先をユーザーに教える返答文を自動生成します。


In [21]:
def get_chatgpt_response(
    user_input: str,
    category_1: str,
    category_2: str,
    template: str,
    model: str = "gpt-3.5-turbo-1106",
    temperature: float = 0,
    max_tokens: int = 500,
):
    """
    ChatGPTに対して対話を投げかけ、返答を取得する
    """
    prompt = template.format(
        user_input=user_input, category_1=category_1, category_2=category_2
    )
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        seed=0
    )
    return response.choices[0].message.content

In [22]:
PROMPT_TEMPLATE = """
    あなたは優秀なカスタマーサポートセンターの担当者です。
    あなたの仕事は下記の顧客の問い合わせとカテゴリーに対して実行手順を処理することで、
    適切な部門のメールアドレスを顧客に伝えるためのメールを作成することです。
    下記の実行手順に従って段階的に考えて、最高の品質でメールの文章を作成してください。

    # 顧客の問い合わせ
     {user_input}

    # カテゴリー
      第一カテゴリー:{category_1}
      第二カテゴリー:{category_2}

    # 実行手順
    1. 第一カテゴリーと第二カテゴリーに応じてメールアドレスを下記から抽出してください。
       下記は第一カテゴリー：第二カテゴリーを表す。
         製品：スペック は product_spec@example.com
         製品：値段 は product_price@example.com
         製品：購入方法 は product_purchase_method@example.com
         注文：注文状況 は order_status@example.com
         注文：注文変更 は order_change@example.com
         注文：支払い方法 は order_payment_method@example.com
         配送：配送ステータス は delivery_status@example.com
         配送：配送オプション は delivery_option@example.com
         配送：配送問題 は delivery_issue@example.com

    2. メール文章を作成
      顧客からの問い合わせに対して、カスタマーセンターの従業員として責任をもって丁寧な文章で
      1で抽出したメールアドレスに問い合わせをお願いする文章を300文字程度で作成する

    # メール文章形式
      考えたメールタイトル
      考えたメール本文
    """

In [23]:
df[:10]

,category_1,category_2,inquiry
0,製品,スペック,このカメラの最大シャッタースピードは何ですか？
1,配送,配送状況,私のパッケージの追跡番号を教えてください。
2,配送,配送オプション,エクスプレス配送は可能ですか？
3,配送,該当なし,特定の日時に配送することはできますか？
4,配送,配送問題,配送された商品が損傷していました。どうすればいいですか？
5,注文,支払い方法,注文の支払いにApple PayやGoogle Payは使用できますか？
6,配送,配送状況,私のパッケージはいつ到着予定ですか？
7,注文,支払い方法,注文の支払いにクレジットカードは使用できますか？
8,注文,注文状況,私の注文はまだ出荷されていませんか？
9,製品,購入方法,この製品はオンラインで注文することができますか？


In [24]:
for values in df.values[:10]:
    print("")
    print("#" * 100)
    category_1, category_2, inquiry = values[0], values[1], values[2]
    print(f"問い合わせ内容:{inquiry}\n")
    response = get_chatgpt_response(
        user_input=inquiry,
        category_1=category_1,
        category_2=category_2,
        template=PROMPT_TEMPLATE,
    )
    print(response)



####################################################################################################
問い合わせ内容:このカメラの最大シャッタースピードは何ですか？

件名：カメラの最大シャッタースピードについての問い合わせ

いつもお世話になっております。カスタマーサポートセンターの担当者です。

お客様からの問い合わせに対して、製品のスペックに関するお問い合わせは product_spec@example.com にお願いいたします。

カメラの最大シャッタースピードについての詳細については、製品のスペックに関する部署がお手伝いさせていただきます。

何かご不明点がございましたら、お気軽にお知らせください。

よろしくお願いいたします。

カスタマーサポートセンター
[会社名]

####################################################################################################
問い合わせ内容:私のパッケージの追跡番号を教えてください。

件名：パッケージの配送状況についてのお問い合わせ

いつも弊社のサービスをご利用いただき、誠にありがとうございます。お客様からのお問い合わせについて、配送状況に関するご質問をお伝えいただき、誠にありがとうございます。

お客様のパッケージの配送状況については、配送カテゴリーに該当いたします。そのため、配送ステータスに関するお問い合わせは、下記のメールアドレスにご連絡いただけますようお願い申し上げます。

配送ステータス：delivery_status@example.com

お手数をおかけいたしますが、上記のメールアドレスに直接お問い合わせいただけますと幸いです。お客様のご理解とご協力に心より感謝申し上げます。

何かご不明点がございましたら、お気軽にお知らせください。引き続き、弊社のサービスをご愛顧いただけますようお願い申し上げます。

敬具

カスタマーサポートセンター

###########################################################################